In [3]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import C2
%matplotlib inline

img_gen=ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)

width=224
height=224
batch_size=4

img=cv2.imread('Tano.JPG')
img=cv2.resize(img,(224,224))
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_origin=img.copy()
img=np.array(img,dtype=np.float32)
img_combine=np.array([img,img,img,img],dtype=np.uint8)
batch_gen=img_gen.flow(img_combine,batch_size=4)
assert next(batch_gen).shape==(batch_size,width,height,3)

plt.figure(figsize=(20,10))
i=1
for batch in batch_gen:
    plt.subplot(1,5,1)
    plt.imshow(img_origin)
    plt.subplot(1,5,i+1)
    plt.imshow(batch[0,:,:,:].astype(np.uint8))
    plt.imshow(batch[1,:,:,:].astype(np.uint8))
    plt.imshow(batch[2,:,:,:].astype(np.uint8))
    plt.imshow(batch[3,:,:,:].astype(np.uint8))
    plt.axis('off')
    i+=1
    if i>4:
        break
        

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\jwen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\jwen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\jwen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\jwen\AppData\Local\Continuum\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\jwen\AppData\Local\Continuum\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module co

TypeError: can only concatenate str (not "list") to str

In [ ]:
train_datagen=ImageDataGenerator(rescale=2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

training_set=train_datagen.flow_from_directory('dataset/training_set',target_size=(64,64),batch_size=32,class_mode='categorical')
test_set=test_datagen.flow_from_directory('dataset/test_set',target_size=(64,64),batch_size=32,class_mode='categorical')

classifier.fit_generator(training_set,steps_per_epoch=250,nb_epoch=25,validation_data=valid_set,validation_steps=63)

In [ ]:
from keras.preprocessing import image as image_utils
test_image=image_utils.load_img('dataset/new_images/new_picture.jpg',target_size=(224,224))
test_image=image_utils.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=classifier.predict_on_batch(test_image)

In [ ]:
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import numpy as np
import cv2

img=cv2.imread('Tano.JPG')
img=cv2.resize(img,(224,224))
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_origin=img.copy()
img=np.array(img,dtype=np.float32)

images=np.random.randint(0,255,(5,224,224,3),dtype=np.uint8)

flipper=iaa.Fliplr(1.0)
images[0]=flipper.augment_image(img)

vflipper=iaa.Flipud(0.4)
images[1]=vflipper.augment_image(img)

blurer=iaa.GaussianBlur(3.0)
images[2]=blurer.augment_image(img)

translater=iaa.Affine(translate_px={'x':-16})
images[3]=translater.augment_image(img)

scaler=iaa.Affine(scale={'y':(0.8,1.2)})
images[4]=scaler.augment_image(img)

i=1
plt.figure(figsize=(20,20))
for image in images:
    plt.subplot(1,6,1)
    plt.imshow(img_origin.astype(np.uint8))
    plt.subplot(1,6,i+1)
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')
    i+=1

In [ ]:
import numpy as np
import imgaug as ia
import imgaup.augmenters as iaa
import matplotlin.pyplot as plt
%matplotlib inline

img=cv2.imread('Tano.JPG')
img=cv2.resize(img,(224,224))
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_origin=img.copy()
img=np.array(img,dtype=np.float32)

img_combine=np.array([img,img,img,img],dtype=np.float32)
sometimes=lambda aug:iaa.Sometimes(0.5,aug)

seq=iaa.Sequential([iaa.Crop(px=(0,16)),iaa.Fliplr(0.4),sometimes(iaa.CropAndPad(
percent=(-0.05,0.1),pad_mode=ia.ALL,pad_cval=(0,255))),sometimes(iaa.Affine(scale={'x':(0.8,1.2),'y':(0.8,1.2)},translate_percent={'x':(-0.2,0.2),'y':(-0.2,0.2)},rotate=(-10,10),shear=(-8,8),order=[0,1],cval=(0,255),model=ia.ALL)),
                   sometimes(iaa.Superpixels(p_replace=(0,1.0),n_segments=(20,200))),
                   sometimes(iaa.Oneof([iaa.GaussianBlur((0,3.0)),iaa.AverageBlur(k=(1,3)),iaa.MedianBlur(k=(3,5)),
                                       sometimes(iaa.Sharpen(alpha=(0,0.2),lightness=(0.1,0.4))),
                                       sometimes(iaa.Emboss(alpha=(0,0.3),strength=(0,0.5))),],random_order=True)
images_aug=seq.augment_images(img_combine)
                             
i=1
plt.figure(figsize=(20,20))
for image in images_aug:
    plt.subplot(1,5,1)
    plt.imshow(img_origin.astype(np.uint8))
    plt.subplot(1,5,i+1)
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')
    i+=1

In [ ]:
from PIL import Image
import os
import pickle
import numpy as np
import cv2
import glob
import pandas as pd
import time
import random
import imgaug as ia
import imgaug.augmenters as iaa

'''隨機改變亮度'''    
class RandomBrightness(object):
    '''Function to randomly make image brighter or darker
    Parameters
    ----------
    delta: float
        the bound of random.uniform distribution
    '''
    def __init__(self, delta=16):
        assert 0 <= delta <= 255
        self.delta = delta

    def __call__(self, image):
        delta = random.uniform(-self.delta, self.delta)
        if random.randint(0, 1):
            image = image + delta
        image = np.clip(image, 0.0, 255.0)
        return image
    
'''隨機改變對比'''
class RandomContrast(object):
    '''Function to strengthen or weaken the contrast in each image
    Parameters
    ----------
    lower: float
        lower bound of random.uniform distribution
    upper: float
        upper bound of random.uniform distribution
    '''
    def __init__(self, lower=0.5, upper=1.5):
        assert upper >= lower, "contrast upper must be >= lower."
        assert lower >= 0, "contrast lower must be non-negative."
        self.lower = lower
        self.upper = upper

    def __call__(self, image):
        alpha = random.uniform(self.lower, self.upper)
        if random.randint(0, 1):
            image = image * alpha
        image = np.clip(image, 0.0, 255.0)
        return image
    
'''包裝所有Augmentation'''
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image):
        for t in self.transforms:
            image= t(image)
        return image

'''包裝Imgaug'''
class ImgAugSequence(object):
    def __init__(self, sequence):
        self.sequence = sequence

    def __call__(self, image):
        image = self.sequence.augment_image(image)
      
        return image
    
class TrainAugmentations(object):
    def __init__(self):
        #Define imgaug.augmenters Sequential transforms
        sometimes = lambda aug: iaa.Sometimes(0.4, aug) # applies the given augmenter in 50% of all cases

        img_seq = iaa.Sequential([
             sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.03*255), per_channel=0.5)),
             sometimes(iaa.ContrastNormalization((0.5, 2.0), per_channel=1),),
             sometimes(iaa.Sharpen(alpha=(0, 0.2), lightness=(0.1, 0.4))), # sharpen images
             sometimes(iaa.Emboss(alpha=(0, 0.3), strength=(0, 0.5))), # emboss images
            ],random_order=True)
        
        self.aug_pipeline = Compose([
            RandomBrightness(16), #make image brighter or darker
            RandomContrast(0.9, 1.1), #strengthen or weaken the contrast in each image
            ImgAugSequence(img_seq),
        ])
        

    def __call__(self, image):
        image= self.aug_pipeline(image)
        return image
    
Augmenation=TrainAugmentations()

In [ ]:
class MaskAugSequence(object):
    def __init__(self, sequence):
        self.sequence = sequence

    def __call__(self, image, mask):
        sequence = self.sequence.to_deterministic() ##用來關閉隨機性
        image = sequence.augment_image(image)
        mask = sequence.augment_image(mask)
        image,  mask= image.astype(np.float32), mask.astype(np.float32)
        return image,  mask